In [3]:
import pandas as pd

In [4]:
access = pd.read_csv("/home/jaime/projet_info_3A/private/access.csv",header=0)
id_api = access["Identifiant"][0]
key_api = access["Clé secrète"][0]

In [5]:
import requests 
import os

In [6]:
import emploi_store

In [7]:
client_id = id_api
client_secret = key_api
access_tokens = {}
cached_packages = None
scope = 'api_offresdemploiv2'

In [8]:
# from https://github.com/Cocorico84/employme/blob/855cfcda2c54f42b9c42dcbf4ac87cdf09248b7f/back/manager.py

In [9]:
import requests


url = "https://api.emploi-store.fr/partenaire"

In [10]:
def get_token():
    url_token = 'https://entreprise.pole-emploi.fr/connexion/oauth2/access_token'
    r = requests.post(url_token, headers={'Content-Type': 'application/x-www-form-urlencoded'},
                      params={'realm': '/partenaire'},
                      data={'grant_type': 'client_credentials', 'client_id': id_api,
                            'client_secret': key_api,
                            'scope': f'application_{id_api} api_offresdemploiv2 o2dsoffre'})
    data = r.json()
    return data['access_token']


In [138]:
def connection(token=get_token(),range_str='0-149'):
    url_offers = f'{url}/offresdemploi/v2/offres/search'
    r = requests.get(url=url_offers,params={'range':range_str},   #{'motsCles': 'informatique'} ,# params={'rantge=':range_str}, #,  params={'motsCles': 'informatique'}
        headers={'Authorization': f'Bearer {get_token()}'})
    data = r.json()

    return data

In [74]:
df.columns

Index([               'accessibleTH',                  'alternance',
                'appellationlibelle',                 'competences',
                           'contact',           'dateActualisation',
                      'dateCreation',                 'description',
               'dureeTravailLibelle', 'dureeTravailLibelleConverti',
                        'entreprise',             'experienceExige',
                 'experienceLibelle',                  'formations',
                                'id',                    'intitule',
                           'langues',               'natureContrat',
                      'nombrePostes',           'qualificationCode',
              'qualificationLibelle',    'qualitesProfessionnelles',
                          'romeCode',                 'romeLibelle',
                           'salaire',             'secteurActivite',
            'secteurActiviteLibelle',         'trancheEffectifEtab',
                       'typeContra

In [115]:
def get_results_as_df(range_str='0-149'):
    
    request_as_dict=connection(get_token(),range_str)

    df = pd.DataFrame.from_dict(request_as_dict['resultats'])
    
#     for col_as_list in ['lieuTravail','origineOffre','partenaires']: #, 'competences', 'contact','entreprise','qualitesProfessionnelles','salaire']:
#         if col_as_list in df.columns:
#             df = pd.concat([df.drop([col_as_list], axis=1), df[col_as_list].apply(pd.Series)], axis=1)
    
    return df #request_as_dict['filtres possibles']


In [124]:
# iteration_index = list(range(0,1150,150))
iteration_index = list(range(0,1000,150))

In [125]:
list_dfs = [get_results_as_df(
    str(str(min(i,1000))+'-'+str(min(i+149,1149)))) for i in list(range(0,1150,150))]

In [126]:
all_dfs = pd.concat(list_dfs)

/home/jaime/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  """Entry point for launching an IPython kernel.


In [127]:
for col_as_list in ['lieuTravail','origineOffre','partenaires', 'competences', 'contact','entreprise','qualitesProfessionnelles','salaire']:
    if col_as_list in all_dfs.columns:
        all_dfs = pd.concat([all_dfs.drop([col_as_list], axis=1), all_dfs[col_as_list].apply(pd.Series)], axis=1)

In [129]:
all_dfs.iloc[0]

accessibleTH                                                               False
alternance                                                                 False
appellationlibelle                             Ouvrier / Ouvrière en viticulture
dateActualisation                                                            NaN
dateCreation                                           2020-04-05T16:06:05+02:00
deplacementCode                                                              NaN
deplacementLibelle                                                           NaN
description                    Nous recherchons une personne pour effectuer l...
dureeTravailLibelle                                         35H Horaires normaux
dureeTravailLibelleConverti                                          Temps plein
experienceCommentaire                                                        NaN
experienceExige                                                                E
experienceLibelle           

In [130]:
all_dfs = all_dfs.astype(str).drop_duplicates()

In [133]:
all_dfs.to_csv("df_max_pages.csv", sep=",")

# loop for more results

In [136]:
request_as_dict=connection(get_token(),'1-5')


In [137]:
request_as_dict

{'filtresPossibles': [{'agregation': [{'nbResultats': 1083,
     'valeurPossible': 'CCE'},
    {'nbResultats': 47054, 'valeurPossible': 'CDD'},
    {'nbResultats': 189959, 'valeurPossible': 'CDI'},
    {'nbResultats': 61, 'valeurPossible': 'CDS'},
    {'nbResultats': 144, 'valeurPossible': 'DDI'},
    {'nbResultats': 67, 'valeurPossible': 'DIN'},
    {'nbResultats': 3599, 'valeurPossible': 'FRA'},
    {'nbResultats': 8276, 'valeurPossible': 'LIB'},
    {'nbResultats': 44747, 'valeurPossible': 'MIS'},
    {'nbResultats': 1150, 'valeurPossible': 'REP'},
    {'nbResultats': 4633, 'valeurPossible': 'SAI'},
    {'nbResultats': 50, 'valeurPossible': 'TTI'}],
   'filtre': 'typeContrat'},
  {'agregation': [{'nbResultats': 39387, 'valeurPossible': '0'},
    {'nbResultats': 142745, 'valeurPossible': '1'},
    {'nbResultats': 92725, 'valeurPossible': '2'},
    {'nbResultats': 25966, 'valeurPossible': '3'}],
   'filtre': 'experience'},
  {'agregation': [{'nbResultats': 107556, 'valeurPossible': '0

In [145]:
token=get_token()
range_str='0-149'
url_offers = f'{url}/offresdemploi/v2/offres/search'
r = requests.get(url=url_offers,params={'range':range_str, 'code':'E1'},   #{'motsCles': 'informatique'} ,# params={'rantge=':range_str}, #,  params={'motsCles': 'informatique'}
    headers={'Authorization': f'Bearer {get_token()}'})
data = r.json()

In [148]:
df = pd.DataFrame.from_dict(data['resultats'])


In [152]:
df['romeCode']

0      A1405
1      I1401
2      E1302
3      C1504
4      J1202
5      H1401
6      K1202
7      M1810
8      I1401
9      H1303
10     M1805
11     K1302
12     K1302
13     K1302
14     D1407
15     F1201
16     F1201
17     M1203
18     M1102
19     M1805
20     H1303
21     D1212
22     M1707
23     I1401
24     M1707
25     M1707
26     M1707
27     C1303
28     M1707
29     F1106
       ...  
120    M1803
121    C1504
122    D1407
123    M1707
124    M1707
125    M1707
126    M1707
127    M1604
128    M1707
129    I1305
130    H1303
131    N1103
132    H1303
133    D1403
134    M1707
135    M1605
136    D1509
137    M1707
138    D1405
139    M1707
140    I1301
141    I1304
142    D1407
143    M1707
144    D1405
145    D1405
146    M1707
147    D1407
148    D1212
149    K1303
Name: romeCode, Length: 150, dtype: object

In [151]:
data['resultats']

[{'accessibleTH': False,
  'alternance': False,
  'appellationlibelle': 'Ouvrier / Ouvrière en viticulture',
  'competences': [{'code': '122825',
    'exigence': 'S',
    'libelle': "Réaliser l'entretien et les soins d'un arbre ou d'un cep"},
   {'code': '122827',
    'exigence': 'S',
    'libelle': "Techniques d'ébourgeonnage"},
   {'code': '122828', 'exigence': 'S', 'libelle': "Techniques d'effeuillage"},
   {'code': '100039',
    'exigence': 'S',
    'libelle': 'Techniques de soins aux arbres ou ceps'},
   {'code': '122826',
    'exigence': 'S',
    'libelle': 'Techniques de taille de fructification'}],
  'contact': {'coordonnees1': 'Courriel : champagne.maletrez.f@orange.fr',
   'courriel': 'champagne.maletrez.f@orange.fr',
   'nom': 'MALETREZ FREDERIC - Mme * MALETREZ'},
  'dateCreation': '2020-04-05T16:06:05+02:00',
  'description': 'Nous recherchons une personne pour effectuer le liage en tache. Vous devez être autonome pour vous rendre dans les vignes.',
  'dureeTravailLibelle'